In [46]:
# import necessary libraries
import requests
from bs4 import BeautifulSoup
from time import sleep
import math
import pandas as pd
from datetime import datetime
import csv

In [47]:
# defining the base url
base_url = 'https://www.airlinequality.com/airline-reviews/'

In [48]:
# for now a focus just a limited focus but will add more airports later
airline = ['klm-royal-dutch-airlines', 'air-china', 'american-airlines']

In [49]:
 # function to generate a list of full_urls of the airlines
def generate_airline_urls(base_url, airline):
    page_urls = []
    for i in airline:
        full_url = base_url + i
        page_urls.append(full_url)
    return page_urls

In [50]:
airline_urls = generate_airline_urls(base_url, airline)

In [51]:
print(airline_urls)

['https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines', 'https://www.airlinequality.com/airline-reviews/air-china', 'https://www.airlinequality.com/airline-reviews/american-airlines']


In [52]:
# function to generate all page urls
def generate_page_urls(airline_url, num_pages):
    page_urls = []
    
    for counter in range(1, num_pages + 1):
        full_url = airline_url + "/page/" + str(counter)
        page_urls.append(full_url)
        
    return page_urls

In [53]:
# Page_url function for KLM
generate_page_urls(airline_urls[0], 123)

['https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/1',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/2',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/3',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/4',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/5',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/6',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/7',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/8',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/9',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/10',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/11',
 'https://www.airlinequality.com/airline-reviews/klm-royal-dutch-airlines/page/12',
 

In [54]:
# function to calculate the page limit for an airline
def page_limit(airline):
    res = requests.get(airline)
    review_source_code = res.text
    soup = BeautifulSoup(review_source_code, 'html.parser')
    
    # this is very complicated code so it needs to be simplified
    text = soup.find(class_="comp comp_reviews-pagination querylist-pagination position-").find("div").get_text()
    clean = text.replace('1 to 10 of ','')
    total_reviews = float(clean.replace(' Reviews',''))
    total_pages = math.ceil(total_reviews/10)

    return print(total_pages)

In [55]:
# page limit function for KLM
page_limit(airline_urls[0])

123


In [56]:
# setting up a function for our scraper
def scrape(page_urls):
    review_data = []
     
    for page_url in page_urls: 
        res = requests.get(page_url)
        soup = BeautifulSoup(res.text, "html.parser")
        reviews = soup.find_all(class_= "body")
        
        for review in reviews:
            review_title = review.find("h2").get_text() # review title
            review_verify = review.find("em").get_text() # verified or not 
            review_published = review.find("time").attrs["datetime"] # date the review is published
            review_data.append({'title': review_title, 'date published': review_published, 'verify_status':  review_verify})
        sleep(1)
    
    return review_data

In [57]:
# scraping the first two pages of KLM reviews
KLM_data = scrape(generate_page_urls(airline_urls[0], 2))
print(KLM_data)

[{'title': '"luggage was never loaded onto the flight"', 'date published': '2021-10-04', 'verify_status': 'Trip Verified'}, {'title': '"What happened with their customer service?"', 'date published': '2021-10-02', 'verify_status': 'Trip Verified'}, {'title': '"staff at the airport were terrible"', 'date published': '2021-09-30', 'verify_status': 'Trip Verified'}, {'title': '"Delightful experience!"', 'date published': '2021-09-27', 'verify_status': 'Not Verified'}, {'title': '"took my small Samsonite hand luggage from me"', 'date published': '2021-09-11', 'verify_status': 'Trip Verified'}, {'title': '"worst flight of our lives"', 'date published': '2021-09-10', 'verify_status': 'Trip Verified'}, {'title': '"experience was very positive"', 'date published': '2021-09-04', 'verify_status': 'Not Verified'}, {'title': '"seats were tiny"', 'date published': '2021-09-02', 'verify_status': 'Trip Verified'}, {'title': '"lied to by a KLM representative who I put my trust in"', 'date published': 

In [44]:
# making a function that will convert the scraped data to a csv file
def convert(airline_data, airline):
    review_data = []

    with open(str(airline) + "_data.csv", "a", encoding = 'utf-8') as csv_file: 
        writer = csv.writer(csv_file, delimiter = ";")
        writer.writerow(["title", "verify_status", "date published"])
    
        for data in airline_data: 
            writer.writerow([data['title'], data['verify_status'], data['date published']])
        
        print("done!")

In [45]:
# Making a csv file for the KLM data. PROBLEM: when opened in excel there are empty rows between each row.
convert(KLM_data, airline[0])

done!


In [ ]:
#### Here I have some problems ####

In [52]:
## I want to extract the review rating but it does not work due to a table at the top of the screen ##
reviews = soup.find_all(class_ = "rating-10")

review_ratings = []
    # create an empty list

for review in reviews:
    review_rating = review.find("span").get_text()
    review_ratings.append(review_rating) 
    
# unfortuneately output also gives the average rating on top of the page 
print(review_ratings)

['\n\t\t\t\t\t\t\t\t\t\t\t\t\t6', '5', '10', '3', '1', '10', '1', '1', '7', '1', '8']


In [144]:
# I am not able to extract reviewer names as we get a weird output
reviews = soup.find_all(class_ = "body")
review_names = []

for review in reviews:
    review_rating = review.find("span").get_text()
    review_rating = review_rating.replace(' ','').replace(' ','') # -> does not work right now
    review_names.append(review_name) 
    
print(review_names)

['\n67 reviews\n\n\n\nMichel Perrin', '\n67 reviews\n\n\n\nMichel Perrin', '\n67 reviews\n\n\n\nMichel Perrin', '\n67 reviews\n\n\n\nMichel Perrin', '\n67 reviews\n\n\n\nMichel Perrin', '\n67 reviews\n\n\n\nMichel Perrin', '\n67 reviews\n\n\n\nMichel Perrin', '\n67 reviews\n\n\n\nMichel Perrin', '\n67 reviews\n\n\n\nMichel Perrin', '\n67 reviews\n\n\n\nMichel Perrin']


In [145]:
# still struggling to get data from the table
data = []

table = review.find('table', attrs={'class':'review-ratings'})
table_body = table.find('tbody')

for row in table.tbody.find_all('tr'):    
    columns = row.find_all('td')

AttributeError: 'NoneType' object has no attribute 'find_all'